# Analyze and plot the amino-acid fitness values

Get variables from `snakemake`:

In [ ]:
if "snakemake" in globals() or "snakemake" in locals():
    # get variables from snakemake
    min_expected_count = snakemake.params.min_expected_count
    clade_corr_min_count = snakemake.params.clade_corr_min_count
    init_ref_clade = snakemake.params.init_ref_clade
    clade_synonyms = snakemake.params.clade_synonyms
    heatmap_minimal_domain = snakemake.params.heatmap_minimal_domain
    orf1ab_to_nsps = snakemake.params.orf1ab_to_nsps
    aamut_all_csv = snakemake.input.aamut_all
    aamut_by_subset_csv = snakemake.input.aamut_by_subset
    aamut_by_clade_csv = snakemake.input.aamut_by_clade
    ref_coding_sites_csv = snakemake.input.ref_coding_sites
    aafitness_csv = snakemake.input.aafitness
    clade_founder_nts_csv = snakemake.input.clade_founder_nts
    outdir = snakemake.output.outdir
else:
    # manually defined if running interactively for debugging
    import yaml

    with open("../config.yaml") as f:
        config = yaml.safe_load(f)

    min_expected_count = config["min_expected_count"]
    clade_corr_min_count = config["clade_corr_min_count"]
    init_ref_clade = config["aa_fitness_init_ref_clade"]
    clade_synonyms = config["clade_synonyms"]
    heatmap_minimal_domain = config["aa_fitness_heatmap_minimal_domain"]
    orf1ab_to_nsps = config["orf1ab_to_nsps"]
    aamut_all_csv = "../results/aa_fitness/aamut_fitness_all.csv"
    aamut_by_subset_csv = "../results/aa_fitness/aamut_fitness_by_subset.csv"
    aamut_by_clade_csv = "../results/aa_fitness/aamut_fitness_by_clade.csv"
    ref_coding_sites_csv = "../results/ref/coding_sites.csv"
    aafitness_csv = "../results/aa_fitness/aa_fitness.csv"
    clade_founder_nts_csv = "../results/clade_founder_nts/clade_founder_nts.csv"
    outdir = "../results/aa_fitness/plots"

Import Python modules:

In [ ]:
import itertools
import os

import altair as alt

import Bio.Seq

import numpy

import pandas as pd

Some settings:

In [ ]:
_ = alt.data_transformers.disable_max_rows()

os.makedirs(outdir, exist_ok=True)

Read input data:

In [ ]:
aamut_all = pd.read_csv(aamut_all_csv)
aamut_by_subset = pd.read_csv(aamut_by_subset_csv)
aamut_by_clade = pd.read_csv(aamut_by_clade_csv)
aafitness = pd.read_csv(aafitness_csv)

Define function that gives clade labels:

In [ ]:
def clade_label(clade):
    if clade in clade_synonyms:
        return f"{clade} ({clade_synonyms[clade]})"
    else:
        return clade

## Correlation in fitness among subsets and clades
Plot correlation in mutation fitness effect ($\Delta f_{xy}$ values) among subsets (geographic locations) and clades with large numbers of counts.
We just plot ORF1ab and not its constituent nsps to avoid double counting.
We only plot amino-acid mutations (not synonymous ones).
We do not include the "all" subset as that contains all the other subsets:

In [ ]:
def plot_corr_scatters(corr_df_tidy):
    "Plot set of correlation scatters."""
    
    subsets = corr_df_tidy["subset"].unique()
    genes = corr_df_tidy["gene"].unique()
    
    corr_df_wide = pd.merge(
        *[
            corr_df_tidy
            .pivot_table(
                index=["gene", "aa_mutation"],
                values=prop,
                columns="subset",
            )
            .reset_index()
            .rename(columns={subset: f"{prop} {subset}" for subset in subsets})
            for prop in ["delta_fitness", "expected_count"]
        ]
    )
    
    delta_fitness_min = corr_df_tidy["delta_fitness"].min()
    delta_fitness_max = corr_df_tidy["delta_fitness"].max()

    gene_selection = alt.selection_multi(
        fields=["gene"], bind="legend",
    )

    expected_count_selection = alt.selection_single(
        bind=alt.binding_range(
            min=1,
            max=min(5 * min_expected_count, corr_df_tidy["expected_count"].quantile(0.9)),
            step=1,
            name="minimum expected count",
        ),
        fields=["cutoff"],
        init={"cutoff": min_expected_count},
    )
    
    highlight = alt.selection_single(
        on="mouseover",
        fields=["gene", "aa_mutation"],
        empty="none",
    )
    
    corr_charts = []
    base_chart = alt.Chart(corr_df_wide)
    for subset1, subset2 in itertools.combinations(subsets, 2):
        base = (
            base_chart
            .encode(
                x=alt.X(
                    f"delta_fitness {subset1}",
                    title=f"{subset1} fitness effect",
                    scale=alt.Scale(domain=(delta_fitness_min, delta_fitness_max), nice=False),
                ),
                y=alt.Y(
                    f"delta_fitness {subset2}",
                    title=f"{subset2} fitness effect",
                    scale=alt.Scale(domain=(delta_fitness_min, delta_fitness_max), nice=False),
                ),
                tooltip=[
                    "gene",
                    "aa_mutation",
                    alt.Tooltip(
                        f"delta_fitness {subset1}", title=f"{subset1} fitness effect",
                    ),
                    alt.Tooltip(
                        f"delta_fitness {subset2}", title=f"{subset2} fitness effect",
                    ),
                    f"expected_count {subset1}",
                    f"expected_count {subset2}",
                ],
            )
            .mark_circle(opacity=0.3)
            .properties(width=200, height=200)
            .transform_filter(gene_selection)
            .transform_filter(
                (alt.datum[f"expected_count {subset1}"] >= expected_count_selection["cutoff"] - 1e-6)
                & (alt.datum[f"expected_count {subset2}"] >= expected_count_selection["cutoff"] - 1e-6)
            )
        )
    
        scatter = (
            base
            .encode(
                color=alt.Color(
                    "gene",
                    scale=alt.Scale(
                        domain=genes,
                        range=["#5778a4"] * len(genes),
                    ),
                    legend=alt.Legend(
                        symbolOpacity=1,
                        orient="bottom",
                        title="click / shift-click to select specific genes to show",
                        titleLimit=500,
                        columns=6,
                    ),
                ),
                size=alt.condition(highlight, alt.value(85), alt.value(30)),
                opacity=alt.condition(highlight, alt.value(1), alt.value(0.3)),
                strokeWidth=alt.condition(highlight, alt.value(1.5), alt.value(0)),
            )
            .mark_circle(stroke="black")
        )
    
        # regression line and correlation coefficient: https://stackoverflow.com/a/60239699
        line = (
            base
            .transform_regression(
                f"delta_fitness {subset1}",
                f"delta_fitness {subset2}",
                extent=(delta_fitness_min, delta_fitness_max),
            )
            .mark_line(color="orange", clip=True)
        )
    
        params_r = (
            base
            .transform_regression(
                f"delta_fitness {subset1}",
                f"delta_fitness {subset2}",
                params=True,
            )
            .transform_calculate(
                r=alt.expr.sqrt(alt.datum["rSquared"]),
                label='"r = " + format(datum.r, ".3f")',
            )
            .mark_text(align="left", color="orange", fontWeight="bold")
            .encode(
                x=alt.value(5),
                y=alt.value(8),
                text=alt.Text("label:N"),
            )
        )
        
        # show number of points
        params_n = (
            base
            .transform_filter(
                (~alt.expr.isNaN(alt.datum[f"delta_fitness {subset1}"]))
                & (~alt.expr.isNaN(alt.datum[f"delta_fitness {subset2}"]))
            )
            .transform_calculate(dummy=alt.datum[f"delta_fitness {subset1}"])
            .transform_aggregate(n="valid(dummy)")
            .transform_calculate(label='"n = " + datum.n')
            .mark_text(align="left", color="orange", fontWeight="bold")
            .encode(
                x=alt.value(5),
                y=alt.value(20),
                text=alt.Text("label:N"),
            )
        )
    
        chart = (
            (scatter + line + params_r + params_n)
            .add_selection(gene_selection)
            .add_selection(expected_count_selection)
            .add_selection(highlight)
        )
    
        corr_charts.append(chart)
    
    ncols = 4
    rows = []
    for i in range(0, len(corr_charts), ncols):
        rows.append(alt.hconcat(*corr_charts[i: i + ncols]))
    corr_chart = alt.vconcat(*rows).configure_axis(grid=False)
    return corr_chart
    
# correlations for subsets
subset_corr_df = (
    aamut_by_subset
    .query("not subset_of_ORF1ab")
    .query("aa_mutation.str[0] != aa_mutation.str[-1]")
    [["subset", "gene", "aa_mutation", "expected_count", "delta_fitness"]]
    .query("subset != 'all'")
)
subset_corr_chart = plot_corr_scatters(subset_corr_df)
subset_corr_chart_file = os.path.join(outdir, "subset_corr_chart.html")
print(f"Saving to {subset_corr_chart_file}\n\n")
subset_corr_chart.save(subset_corr_chart_file)
display(subset_corr_chart)

# correlations for clades
clade_corr_df = (
    aamut_by_clade
    .query("not subset_of_ORF1ab")
    .query("aa_mutation.str[0] != aa_mutation.str[-1]")
    [["clade", "gene", "aa_mutation", "expected_count", "delta_fitness"]]
    .assign(
        clade_counts=lambda x: x.groupby("clade")["expected_count"].transform("sum"),
        clade=lambda x: x["clade"].map(clade_label).str.replace(".", "_", regex=False),
    )
    .query("clade_counts >= @clade_corr_min_count")
    .drop(columns="clade_counts")
    .rename(columns={"clade": "subset"})
)
clade_corr_chart = plot_corr_scatters(clade_corr_df)
clade_corr_chart_file = os.path.join(outdir, "clade_corr_chart.html")
print(f"Saving to {clade_corr_chart_file}")
clade_corr_chart.save(clade_corr_chart_file)
display(clade_corr_chart)

## Correlations among different gene subsets
Correlations of mutation effects among clades among different gene subsets using a few different cutoffs for expected counts.
We compute the average correlation across all pairs of distinct clades with adequate counts:

In [ ]:
count_thresholds = [
    min_expected_count // 2,
    min_expected_count,
    min_expected_count * 2,
    min_expected_count * 4,
]

def get_subset_corrs(df):
    """Average correlations of pairs of distinct subsets for spike/non-spike mutations."""
    return (
        pd.concat(
            [
            df.query("expected_count >= @n").assign(count_threshold=n)
            for n in count_thresholds
            ]
        )
        .pivot_table(
            index=["count_threshold", "gene", "aa_mutation"],
            columns="subset",
            values="delta_fitness",
        )
        .reset_index()
        .assign(
            protein=lambda x: numpy.where(
                x["gene"] == "S", "spike", "non-spike",
            ),
        )
        .groupby(["count_threshold", "protein"])
        .corr()
        .reset_index()
        .melt(
            id_vars=["count_threshold", "protein", "subset"],
            value_name="correlation",
            var_name="subset_2",
        )
        .query("subset != subset_2")
    )

subset_corrs = (
    pd.concat(
        [
            get_subset_corrs(clade_corr_df).assign(subset_type="subset by clade"),
            get_subset_corrs(subset_corr_df).assign(subset_type="subset by geography"),
        ]
    )
    .groupby(["subset_type", "count_threshold", "protein"], as_index=False)
    .aggregate({"correlation": "mean"})
)

subset_corrs_chart = (
    alt.Chart(subset_corrs)
    .encode(
        x=alt.X("count_threshold:N", title="min expected counts"),
        y=alt.Y(
            "correlation",
            title="average correlation among subsets",
            scale=alt.Scale(domain=(0, 1)),
        ),
        color=alt.Color("protein", scale=alt.Scale(range=["#999999", "#D55E00"])),
        shape=alt.Shape("protein"),
        fill=alt.Fill("protein", scale=alt.Scale(range=["#999999", "white"])),
        fillOpacity=alt.Fill("protein", scale=alt.Scale(range=[0.6, 0])),
        column=alt.Column(
            "subset_type",
            title=None,
            spacing=20,
            header=alt.Header(labelFontSize=11, labelFontWeight="bold"),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3f") if subset_corrs[c].dtype == float else c
            for c in subset_corrs.columns
        ],
    )
    .mark_point(size=50)
    .properties(height=190, width=alt.Step(25))
)

subset_corrs_chart_file = os.path.join(outdir, "spike_nonspike_corrs.html")
print(f"Saving to {subset_corrs_chart_file}")
subset_corrs_chart.save(subset_corrs_chart_file)

subset_corrs_chart

Correlations among clades by amino-acid divergence.
First get clade founder amino acids:

In [ ]:
# codon translation table
codon_table = {
    f"{nt1}{nt2}{nt3}": str(Bio.Seq.Seq(f"{nt1}{nt2}{nt3}").translate())
    for nt1 in "ACGT" for nt2 in "ACGT" for nt3 in "ACGT"
}

# get clade founder amino-acids
clade_founder_aas = (
    pd.read_csv(clade_founder_nts_csv)
    .query("gene != 'noncoding'")
    [["clade", "gene", "codon", "codon_site"]]
    .drop_duplicates()
    .assign(
        gene=lambda x: x["gene"].str.split(";"),
        codon=lambda x: x["codon"].str.split(";"),
        codon_site=lambda x: x["codon_site"].str.split(";"),
    )
    .explode(["gene", "codon", "codon_site"])
    .assign(
        aa=lambda x: x["codon"].map(codon_table),
        codon_site=lambda x: x["codon_site"].astype(int),
        clade=lambda x: x["clade"].map(clade_label),
    )
    .rename(columns={"codon_site": "site", "aa": "amino acid"})
    .drop(columns="codon")
    .query("gene != 'ORF1a'")  # this is just subset of ORF1ab
)

Now compute amino-acid divergence among spike and non-spike proteins:

In [ ]:
aa_divergences = []
for c1, c2 in itertools.combinations(clade_corr_df["subset"].unique(), 2):
    aa_divergences.append(
        clade_founder_aas
        .assign(
            clade=lambda x: x["clade"].str.replace(".", "_", regex=False),
            protein=lambda x: numpy.where(
                x["gene"] == "S", "spike", "non-spike",
            ),
        )
        .query("(clade == @c1) or (clade == @c2)")
        .pivot_table(
            index=["protein", "site"], columns="clade", aggfunc="first", values="amino acid",
        )
        .reset_index()
        .assign(ident=lambda x: x[c1] == x[c2])
        .groupby("protein", as_index=False)
        .aggregate(
            n=pd.NamedAgg("site", "count"),
            n_ident=pd.NamedAgg("ident", "sum"),
        )
        .assign(
            percent_divergence=lambda x: 100 * (1 - x["n_ident"] / x["n"]),
            clade_1=c1,
            clade_2=c2
        )
    )

corrs_by_cladediv_chart_file = os.path.join(outdir, "spike_nonspike_corrs_by_divergence.html")

if len(aa_divergences):
    aa_divergences = pd.concat(
        [
            pd.concat(aa_divergences, ignore_index=True),
            pd.concat(aa_divergences, ignore_index=True).rename(
                columns={"clade_1": "clade_2", "clade_2": "clade_1"}
            ),
        ]
    )

    # Combine the amino-acid divergences with the clade-to-clade correlations:
    corrs_by_cladediv = (
        get_subset_corrs(clade_corr_df)
        .rename(columns={"subset": "clade_1", "subset_2": "clade_2"})
        .merge(
            aa_divergences[["protein", "clade_1", "clade_2", "percent_divergence"]],
            on=["clade_1", "clade_2", "protein"],
            how="left",
            validate="many_to_one",
        )
    )

    assert corrs_by_cladediv.notnull().all().all()

    # Now plot correlation in mutation effects by percent protein divergence:
    count_threshold_selection = alt.selection_single(
        fields=["count_threshold"],
        bind=alt.binding_radio(
            options=corrs_by_cladediv["count_threshold"].unique(),
            name="minimum expected counts",
        ),
        init={"count_threshold": min_expected_count},
    )

    corrs_by_cladediv_chart = (
        alt.Chart(corrs_by_cladediv)
        .encode(
            x=alt.X(
                "percent_divergence",
                axis=alt.Axis(grid=False),
                title="% protein divergence between clades",
            ),
            y=alt.Y(
                "correlation",
                title="correlation between clade pairs",
                scale=alt.Scale(domain=(0, 1)),
            ),
            color=alt.Color("protein", scale=alt.Scale(range=["#999999", "#D55E00"])),
            shape=alt.Shape("protein"),
            fill=alt.Fill("protein", scale=alt.Scale(range=["#999999", "white"])),
            fillOpacity=alt.Fill("protein", scale=alt.Scale(range=[0.6, 0])),
            tooltip=[
                alt.Tooltip(c, format=".3f") if corrs_by_cladediv[c].dtype == float else c
                for c in corrs_by_cladediv.columns
            ],
        )
        .mark_point(size=50)
        .add_selection(count_threshold_selection)
        .transform_filter(count_threshold_selection)
        .properties(width=190, height=190)
    )

    print(f"Saving to {corrs_by_cladediv_chart_file}")
    corrs_by_cladediv_chart.save(corrs_by_cladediv_chart_file)

    display(corrs_by_cladediv_chart)

else:
    print(f"Creating empty file {corrs_by_cladediv_chart_file}")
    (
        alt.Chart(pd.DataFrame({"x": [0.5], "y": [0.5], "text": "empty chart"}))
        .encode(x="x", y="y", text="text")
        .mark_text()
        .save(corrs_by_cladediv_chart_file)
    )

## Histograms of mutation effects
Histograms of mutation effects.
We make a version both with ORF1ab labeled genes and nsp labeled genes:

In [ ]:
mut_type_order = ["synonymous", "nonsynonymous", "stop"]
mut_type_colors = ["#009E73", "#56B4E9", "#E69F00"]

gene_selection = alt.selection_multi(
    fields=["gene"], bind="legend",
)

expected_count_selection = alt.selection_single(
    bind=alt.binding_range(
        min=1,
        max=min(5 * min_expected_count, aamut_all["expected_count"].quantile(0.9)),
        step=1,
        name="minimum expected count",
    ),
    fields=["cutoff"],
    init={"cutoff": min_expected_count},
)

namings = ["nsp"]  # can also be ["nsp", "ORF1ab"] to plot ORF1ab naming

for orf1ab_nsp in namings:
    query_str = (
        "not subset_of_ORF1ab" if orf1ab_nsp == "ORF1ab"
        else "gene != 'ORF1ab'"
    )
    
    hist_df = (
        aamut_all
        .query(query_str)
        .assign(
            mut_type=lambda x: x["aa_mutation"].map(
                lambda m: (
                    "synonymous" if m[0] == m[-1]
                    else "stop" if m[-1] == "*" else "nonsynonymous"
                )
            )
        )
        [["gene", "expected_count", "delta_fitness", "mut_type"]]
    )
    
    delta_fitness_max = hist_df["delta_fitness"].max()
    delta_fitness_min = hist_df["delta_fitness"].min()

    hist_chart = (
        alt.Chart(hist_df)
        .encode(
            x=alt.X(
                "delta_fitness",
                bin=alt.Bin(step=0.5),
                scale=alt.Scale(domain=(delta_fitness_min, delta_fitness_max)),
                axis=alt.Axis(
                    values=[
                        2 * t
                        for t in range(
                            int(delta_fitness_min // 2), int(delta_fitness_max // 2)
                        )
                    ]
                ),
                title="fitness effect of mutation",
            ),
            y=alt.Y("count()", title="number of mutations"),
            color=alt.Color(
                "mut_type", scale=alt.Scale(range=mut_type_colors), legend=None, sort=mut_type_order,
            ),
            stroke=alt.Stroke(
                "mut_type", scale=alt.Scale(range=mut_type_colors), legend=None, sort=mut_type_order,
            ),
            opacity=alt.Opacity(
                "gene",
                scale=alt.Scale(
                    domain=hist_df["gene"].unique(),
                    range=[1] * hist_df["gene"].nunique(),
                ),
                legend=alt.Legend(
                    symbolFillColor="darkgray",
                    orient="bottom",
                    title="click / shift-click to select specific genes to show",
                    titleLimit=500,
                    columns=13,
                    padding=5,
                ),
            ),    
            facet=alt.Facet(
                "mut_type",
                title=None,
                columns=len(mut_type_order),
                header=alt.Header(labelFontSize=12, labelFontWeight="bold", labelPadding=2),
                spacing=8,
                sort=mut_type_order,
            ),
        )
        .mark_bar(clip=True)
        .transform_filter(gene_selection)
        .transform_filter(
            alt.datum[f"expected_count"] >= expected_count_selection["cutoff"] - 1e-6
        )
        .add_selection(gene_selection)
        .add_selection(expected_count_selection)
        .properties(width=235, height=125)
        .resolve_scale(y="independent")
        .configure_axis(grid=False)
    )
    
    if len(namings) > 1:
        chartfile = os.path.join(outdir, f"effects_histogram_{orf1ab_nsp}_naming.html")
    else:
        chartfile = os.path.join(outdir, "effects_histogram.html")
    print(f"Saving to {chartfile}")
    hist_chart.save(chartfile)

    display(hist_chart)

## Distributions of mutation effects across genes
This plot summarizes the above histograms for all genes in a single plot, showing the individual nsps (not ORF1ab).

For one version of the plot, we order genes by position in the genome, so first get that order:

In [ ]:
# get order of all genes after ORF1ab
gene_position_order = (
    pd.read_csv(ref_coding_sites_csv)
    [["gene"]]
    .drop_duplicates()
    .assign(gene=lambda x: x["gene"].str.split(";"))
    .explode("gene")
    .query("gene not in ['noncoding', 'ORF1a', 'ORF1ab']")
    .drop_duplicates()
    ["gene"]
    .tolist()
)

# add nsp order
gene_position_order = list(orf1ab_to_nsps) + gene_position_order

Now make the plots:

In [ ]:
dist_df = (
    aamut_all
    .query("gene != 'ORF1ab'")
    .assign(
        mut_type=lambda x: x["aa_mutation"].map(
            lambda m: (
                "synonymous" if m[0] == m[-1]
                else "stop" if m[-1] == "*" else "nonsynonymous"
            )
        )
    )
    [["gene", "expected_count", "delta_fitness", "mut_type"]]
)

# order by median effect of nonsynonymous mutations
gene_order = (
    dist_df
    .query("mut_type == 'nonsynonymous'")
    .query("expected_count >= @min_expected_count")
    .groupby("gene", as_index=False)
    .aggregate({"delta_fitness": "median"})
    .sort_values("delta_fitness")
    ["gene"]
    .tolist()
)

mut_type_selection = alt.selection_multi(
    fields=["mut_type"], bind="legend", toggle="true",
)

base_dist_chart = (
    alt.Chart(dist_df)
    .transform_filter(
        alt.datum[f"expected_count"] >= expected_count_selection["cutoff"] - 1e-6
    )
    .transform_aggregate(
        median="median(delta_fitness)",
        lower_quartile="q1(delta_fitness)",
        upper_quartile="q3(delta_fitness)",
        n="valid(delta_fitness)",
        groupby=["gene", "mut_type"],
    )
    .transform_calculate(
        medianplus=alt.datum["median"] + 0.1,
        medianminus=alt.datum["median"] - 0.1,
    )
    .encode(
        x=alt.X(
            "mut_type",
            title=None,
            axis=alt.Axis(labels=False, ticks=True, values=[mut_type_order[1]]),
            scale=alt.Scale(
                # dummy added for space between samples
                domain=["dummy", *mut_type_order, "dummy2"]),
        ),
        color=alt.Color(
            "mut_type",
            title="mutation type",
            legend=alt.Legend(orient="bottom", titleOrient="left"),
            scale=alt.Scale(range=mut_type_colors),
            sort=mut_type_order,
        ),
        opacity=alt.condition(mut_type_selection, alt.value(1), alt.value(0.2)),
        tooltip=[
            "gene",
            alt.Tooltip("mut_type", title="mutation type"),
            alt.Tooltip("median:Q", format=".2f"),
            alt.Tooltip("lower_quartile:Q", format=".2f"),
            alt.Tooltip("upper_quartile:Q", format=".2f"),
            alt.Tooltip("n:Q", title="number of mutations"),
        ],
    )
)

barwidth = 6

median_dist_chart = (
    base_dist_chart
    .encode(
        y=alt.Y("medianplus:Q", title="fitness effect of mutation"),
        y2=alt.Y2("medianminus:Q", title="fitness effect of mutation"),
    )
    .mark_bar(size=barwidth, filled=True)
)

bars_dist_chart = (
    base_dist_chart
    .encode(
        y="lower_quartile:Q",
        y2="upper_quartile:Q",
    )
    .mark_bar(size=barwidth, filled=True, fillOpacity=0.5, strokeWidth=0)
)

zero_line = (
   alt.Chart(pd.DataFrame({"zero": [0]}))
    .encode(y="zero:Q")
    .mark_rule(color="gray", strokeDash=[3, 3])
)

height = 200

for genesort, chartfile in [
    (gene_order, os.path.join(outdir, "effects_dist.html")),
    (gene_position_order, os.path.join(outdir, "effects_dist_position_order.html")),
]:
    dist_chart = (
        (bars_dist_chart + median_dist_chart + zero_line)
        .properties(height=height, width=alt.Step(barwidth))
        .facet(
            column=alt.Column(
                "gene",
                sort=genesort,
                title=None,
                header=alt.Header(
                    labelOrient="bottom",
                    labelAngle=-90,
                    labelAlign="right",
                    labelBaseline="middle",
                    labelPadding=height + 10,
                ),
            ),
            spacing=0,
        )
        .add_selection(expected_count_selection)
        .add_selection(mut_type_selection)
        .configure_axis(grid=False)
        .configure_view(stroke=None)
    )

    print(f"\nSaving to {chartfile}")
    dist_chart.save(chartfile)

    display(dist_chart)

## Plot results for individual genes
Now we plot heatmaps of the amino-acid fitness estimates for each protein.

First, get the "wildtype" amino acid at each site in our reference sequence:

In [ ]:
# now convert ORF1ab numbers to nsp numbers
orf1ab_to_nsps_df = pd.concat(
    [
        pd.DataFrame(
            [(i, i - start + 1) for i in range(start, end + 1)],
            columns=["ORF1ab_site", "nsp_site"],
        ).assign(nsp=nsp).drop_duplicates()
        for nsp, (start, end) in orf1ab_to_nsps.items()
    ],
    ignore_index=True,
)

clade_founder_aas = pd.concat(
    [
        clade_founder_aas,
        (
            clade_founder_aas
            .query("gene == 'ORF1ab'")
            .merge(
                orf1ab_to_nsps_df,
                left_on="site",
                right_on="ORF1ab_site",
                validate="many_to_one",
            )
            .drop(columns=["gene", "ORF1ab_site", "site"])
            .rename(columns={"nsp_site": "site", "nsp": "gene"})
        ),
    ],
    ignore_index=False,
)

Now plot the heatmaps:

In [ ]:
def plot_aa_fitness(gene, fitness_df, clade_founder_df):
    """Plot of amino-acid fitness values."""
    
    # biochemically ordered alphabet
    aas = tuple("RKHDEQNSTYWFAILMVGPC*")
    assert set(fitness_df["amino acid"]).issubset(aas)
    
    sites = fitness_df["site"].unique().tolist()
    
    expected_count_selection = alt.selection_single(
        bind=alt.binding_range(
            min=1,
            max=min(5 * min_expected_count, fitness_df["expected_count"].quantile(0.9)),
            step=1,
            name="minimum expected count",
        ),
        fields=["cutoff"],
        init={"cutoff": min_expected_count},
    )
   
    site_zoom_brush = alt.selection_interval(
        encodings=["x"],
        mark=alt.BrushConfig(
            stroke="gold", strokeWidth=1.5, fill="yellow", fillOpacity=0.3,
        ),
    )
        
    base = (
        alt.Chart(fitness_df)
        .encode(x=alt.X("site:O", axis=alt.Axis(labelOverlap="parity")))
        .transform_filter(
            alt.datum[f"expected_count"] >= expected_count_selection["cutoff"] - 1e-6
        )
    )
    
    heatmap_y = alt.Y("amino acid", sort=aas, scale=alt.Scale(domain=aas))
    heatmap_base = (
        base
        .encode(y=heatmap_y)
        .properties(width=alt.Step(12), height=alt.Step(12))
    )
    
    # background fill for missing values in heatmap, imputing dummy stat
    # to get all cells
    heatmap_bg = (
        heatmap_base
        .transform_impute(
            impute="_stat_dummy",
            key="amino acid",
            keyvals=aas,
            groupby=["site"],
            value=None,
        )
        .mark_rect(color="gray", opacity=0.25)
    )
    
    # heatmap showing non-filtered amino acids
    heatmap_aas = (
        heatmap_base
        .encode(
            color=alt.Color(
                "fitness:Q",
                legend=alt.Legend(
                    orient="bottom",
                    titleOrient="left",
                    gradientLength=150,
                    gradientStrokeColor="black",
                    gradientStrokeWidth=0.5,
                ),
                scale=alt.Scale(
                    zero=True,
                    nice=False,
                    type="linear",
                    domainMid=0,
                    domain=alt.DomainUnionWith(heatmap_minimal_domain),
                ),
            ),
            stroke=alt.value("black"),
            tooltip=[
                alt.Tooltip(c, format=".3g")
                if fitness_df[c].dtype == float
                else c
                for c in fitness_df.columns
            ],
        )
        .mark_rect()
    )
    
    # place X values at "wildtype"
    wildtype_clade_selection = alt.selection_single(
        fields=["clade"],
        bind=alt.binding_select(
            options=clade_founder_df["clade"].unique(),
            name="X denotes wildtype in",
        ),
        init={"clade": clade_label(init_ref_clade)},
    )
    heatmap_wildtype = (
        alt.Chart(clade_founder_df.query("site in @sites"))
        .encode(
            x=alt.X("site:O"),
            y=heatmap_y,
        )
        .mark_text(text="x", color="black")
        .add_selection(wildtype_clade_selection)
        .transform_filter(wildtype_clade_selection)
        .transform_filter(site_zoom_brush)
    )

    heatmap = (
        (heatmap_bg + heatmap_aas + heatmap_wildtype)
        .add_selection(expected_count_selection)
        .transform_filter(site_zoom_brush)
    )
    
    # make lineplot
    site_statistics = ["mean", "max", "min"]
    site_stat = alt.selection_single(
        bind=alt.binding_radio(
            options=site_statistics,
            name="site fitness statistic",
        ),
        fields=["site fitness statistic"],
        init={"site fitness statistic": site_statistics[0]},
    )
    
    lineplot = (
        base
        .transform_filter(alt.datum["amino acid"] != "*")
        .transform_aggregate(
            **{stat: f"{stat}(fitness)" for stat in site_statistics},
            groupby=["site"],
        )
        .transform_fold(
            site_statistics,
            ["site fitness statistic", "site fitness"],
        )
        .add_selection(site_stat)
        .add_selection(site_zoom_brush)
        .transform_filter(site_stat)
        .encode(
            y=alt.Y("site fitness:Q", axis=alt.Axis(grid=False)),
            tooltip=[
                "site",
                alt.Tooltip("site fitness:Q", format=".3g"),
                "site fitness statistic:N",
            ],
        )
        .mark_area(color="black", opacity=0.7)
        .properties(
            height=75,
            width=min(750, 12 * fitness_df["site"].nunique()),
            title=alt.TitleParams(
                "use this site plot to zoom into regions on the heat map",
                anchor="start",
                fontWeight="normal",
                fontSize=11,
            ),
        )
    )
    
    show_stop = alt.selection_single(
        fields=["_dummy"],
        bind=alt.binding_radio(
            options=["yes", "no"],
            name="show stop in magenta on top site plot",
        ),
        init={"_dummy": "no"},
    )
    
    stopplot = (
        base
        .add_selection(show_stop)
        .transform_filter(alt.datum["amino acid"] == "*")
        .transform_calculate(_dummy="'yes'")
        .transform_filter(show_stop)
        .encode(
            y=alt.Y("fitness", title="site fitness"),
            color=alt.value("#CC79A7"),
            tooltip=["site", alt.Tooltip("fitness", format=".3g", title="stop fitness")],
        )
        .mark_line(point=True, strokeWidth=0.5, strokeDash=[2, 2])
    )
    
    return (
        (alt.layer(lineplot, stopplot) & heatmap)
        .properties(
            title=alt.TitleParams(
                f"estimated fitness of amino acids for SARS-CoV-2 {gene} protein",
                fontSize=15,
            ),
        )
        .resolve_scale(color="independent")
    )


for gene, fitness_df in (
    aafitness
    [["gene", "aa_site", "aa", "fitness", "expected_count"]]
    .rename(columns={"aa_site": "site", "aa": "amino acid"})
    .groupby("gene")
):
    if gene == "ORF1ab":
        continue  # do not make a plot for ORF1ab, too big
    chart = plot_aa_fitness(gene, fitness_df, clade_founder_aas.query("gene == @gene"))
    gene_name_for_file = gene.replace(" ", "_").replace("(", "").replace(")", "")
    chartfile = os.path.join(outdir, f"{gene.split()[0]}.html")
    print(f"\nSaving chart for {gene} to {chartfile}")
    chart.save(chartfile)
    display(chart)